## 연습문제 - 201810803 장서령

주의:

 - 다른 과제보다 많이 약 10% 배점합니다

 - 다른 학생과 협력하지 마세요.

 - 제한시간이 다른 과제에 비해 짧아요 (3.5일)



A, B 2인의 가위바위보 후 내기금액을 이긴 사람에게 지급하는 게임을 블록체인에 개발하세요.

게임 컨트랙은 Rsp, 구현할 함수는 다음과 같다. 그 외 필요로 하는 함수는 추가할 수 있다.



함수 | 설명

-----|-----

setA | 무작위로 가위, 바위, 보를 생성하고, 설정한다. ```1000 wei```씩 금액을 건다.

setB | 외부에서 직접 입력하여 설정한다. ```1000 wei```씩 금액을 건다.

play | 가위, 바위, 보의 승패를 정한다.

distributeBetAmount | 이긴 측에게 내기 금액을 분배한다. A가 이기면 B의 내기 금액 ```1000 wei```를 가지게 된다.

getMatchResult() | "A wins", "B wins", "tie" 결과를 출력



노드에서 게임을 다음과 같이 진행하세요.

- 게임 전의 A잔고, B잔고, 컨트랙 잔고 출력

- A가 내기금액 걸고, 컴퓨터에서 무작위로 선택한 가위바위보

- B가 내기금액 걸고, 사용자가 입력한 가위바위보

- 승부내기

- 누가 이겼는지 출력

- 게임 끝나고 A잔고, B잔고, 컨트랙 잔고 출력

## 1단계 : 컨트랙 개발

In [3]:
%%writefile src/Rsp.sol
pragma solidity ^0.5.0;

contract Rsp { //32bit짜리 숫자를 생성하는 rand함수 만들기
    uint8 randA;
    uint8 randB;
    uint8 whatresult;
    string transresult="";
    address owner; //컨트랙 제작자
    mapping (address => uint) balanceOf; // 잔고를 저장하는 mapping. 잔고의 key는 address, address의 key는 uint
    constructor() public { // 생성자, 누구나 객체를 만들 수 있음
        owner = msg.sender;  
    }
    
    //무작위로 가위, 바위, 보를 생성하고, 설정한다. 1000 wei씩 금액을 건다.
    function setA(uint amount) payable public {
        require(msg.value == amount);
        balanceOf[msg.sender] += amount;
        randA = uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3);
        // 0 : 가위, 1 : 바위, 2 : 보
    } 

    //외부에서 직접 입력하여 설정한다. 1000 wei씩 금액을 건다.
    function setB(address addr, uint amount, uint8 _value) payable public {
        require(msg.value == amount);
        balanceOf[addr] += amount;
        randB = _value;
    }
    
    //가위, 바위, 보의 승패를 정한다.
    function play() public { //tie = 3, A win = 4, B win = 5
        if(randA==0) { //가위
            if(randB==0) { //가위
                whatresult= 3;
            }
            else if(randB==1){ //바위
                whatresult= 5;
            }
            else{ //B가 보
                whatresult= 4;
            }
        }
        else if(randA == 1) { //바위
            if(randB==0) { //가위
                whatresult= 4;
            }
            else if(randB==1) {
                whatresult= 3;
            }
            else {
                whatresult= 5;
            }
        }
        else if(randA==2){ //보
            if(randB==0){ //가위
                whatresult= 5;
            }
            else if(randB==1){
                whatresult= 4;
            }
            else {
                whatresult= 3;
            }
        }
        if(whatresult==3){
            transresult = "tie";
        }
        else if(whatresult==4){
            transresult = "A win";
        }
        else if(whatresult==5){
            transresult = "B win";
        }
    }
    
    //이긴 측에게 내기 금액을 분배한다. A가 이기면 B의 내기 금액 1000 wei를 가지게 된다.
    function distributeBetAmount(address payable receiver) public payable{
        if(whatresult==3){ //tie
            balanceOf[receiver];
            balanceOf[msg.sender];
        }
        else if(whatresult==4){ //A win
            balanceOf[receiver] -= 1000;
            balanceOf[msg.sender] += 1000;
        }
        else if(whatresult==5){ //B win
            balanceOf[msg.sender] -= 1000;
            balanceOf[receiver] += 1000;
        }
    }
    
    // 컨트랙 잔고 확인
    function getContract() public view returns(uint){
         return (address(this).balance);
    }
    
    //A의 잔고 확인
    function getA() public view returns(uint) {
        return (balanceOf[owner]); // A의 잔고 출력       
    }
    
    //B의 잔고 확인
    function getB(address addr) public view returns(uint) {
        return balanceOf[addr]; //B의 계좌 잔고를 읽어옴
    }
    
    //"A wins", "B wins", "tie" 결과를 출력
    function getMatchResult()public view returns(string memory) {
        return (transresult);
    }
    
    function getwhatresult() public view returns(uint) {
        return (whatresult);
    }
    
}

Overwriting src/Rsp.sol


## 2단계 : 컴파일

In [4]:
!solc src/Rsp.sol --combined-json abi,bin > src/Rsp.json

## 3단계 : 배포

In [5]:
%%writefile src/RspDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/Rsp.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
var _abiArray = JSON.parse(_json.contracts["src/Rsp.sol:Rsp"].abi);
//var _bin = _json.contracts.sHello2.bin;
var _bin="0x" + _json.contracts["src/Rsp.sol:Rsp"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);

    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas); //gas를 산정하는 함수를 위에 넣어놓음 
    });

    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 791305}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/RspDeploy.js


In [6]:
!node src/RspDeploy.js

Deploying the contract from 0x284b80A9B31d519A9eDE1902eabB0724258d7f64
>> gas: 733695
hash: 0x55b8ff5e6bddb5761b8e8cf5f8c153ea7286c2d4e096626bf7253e98d30881eb
---> The contract deployed to: 0x83Ca6fB049621290ACe825472054FF00eB017998


## 4단계 : 사용

In [7]:
%%writefile src/RspUse.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));       //ok
var fs=require('fs');
var _str = fs.readFileSync("src/Rsp.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
var _abiArray = JSON.parse(_json.contracts["src/Rsp.sol:Rsp"].abi);

async function doIt() {//arr 대신 _rspGame
    var _rspGame = new web3.eth.Contract(_abiArray, "0x83Ca6fB049621290ACe825472054FF00eB017998");
    const accounts = await web3.eth.getAccounts();
    console.log("<The Init Balance>"); //A,B,컨트랙 초기잔고 출력 
    const Ainit = await _rspGame.methods.getA().call();           
    const Binit = await _rspGame.methods.getB(accounts[1]).call(); 
    const Cinit = await _rspGame.methods.getContract().call();
    console.log("A : " + Ainit);
    console.log("Contract : " + Cinit);
    console.log("B : " + Binit);
    console.log("setA, setB---------");
    console.log("\n");
    await _rspGame.methods.setA("1000").send({from: accounts[0], gas:135482,value:1000}); //A 가위바위보
    await _rspGame.methods.setB(accounts[1],"1000","1").send({from: accounts[0], gas:135482,value:1000}); //B 가위바위보
    const A = await _rspGame.methods.getA().call();           
    const B = await _rspGame.methods.getB(accounts[1]).call(); 
    const C = await _rspGame.methods.getContract().call();
    console.log("<The Balance Before The Game>"); //A,B,컨트랙 게임 전 잔고 출력 
    console.log("A : " + A);
    //console.log("Contract : " + C);
    console.log("B : " + B);
    
    console.log("play----------------");
    await _rspGame.methods.play().send({from: accounts[0], gas:135482}); //승부내기
    console.log("\n");
    console.log("<The Match Result>");
    const Mresult = await _rspGame.methods.getMatchResult().call();
    console.log("result is... " + Mresult); //이긴사람 출력  
    const what = await _rspGame.methods.getwhatresult().call();   // 
    console.log("whatresult is "+what); //                  
    console.log("send the money-------"); 
    await _rspGame.methods.distributeBetAmount(accounts[1]).send({from: accounts[0], gas:135482});
    console.log("\n");
                       
    console.log("<The Last Balance>");
    const AafterBal = await _rspGame.methods.getA().call();
    const ContractafterBal = await _rspGame.methods.getB(accounts[1]).call(); 
    const BafterBal = await _rspGame.methods.getContract().call();
    console.log("A : " + AafterBal);
    console.log("Contract : " + ContractafterBal);
    console.log("B : " + BafterBal);
}

doIt()

Overwriting src/RspUse.js


In [8]:
!node src/RspUse.js

<The Init Balance>
A : 0
Contract : 0
B : 0
setA, setB---------


<The Balance Before The Game>
A : 1000
B : 1000
play----------------


<The Match Result>
result is... tie
whatresult is 3
send the money-------


<The Last Balance>
A : 1000
Contract : 1000
B : 2000


####  A와 컨트랙이 다르다고 생각하고 풀었다. 실습했던 Bankv4에 대한 내용이 있는 강의자료를 정독하면서 풀었다. owner의 잔고가 변하는 반면 컨트랙 잔고의 변동은 없다는 표의 내용을 보고 A와 컨트랙이 다르다고 생각했다.

3일 내내 geth를 이용해서 문제를 해결하려고 했으나, out of gas와 ABI 문제가 발생한다고 계속 오류가 떠서 결국 가나슈로 과제를 해결했다. geth를 이용하면 배포단계까지는 성공하나, 계속 컨트랙 사용 단계에서 오류가 발생했다. 리믹스 코드를 돌려보며 오류가 있는지 없는지도 확인해보았고, 또 리믹스로 함수를 실행할 때마다 소모된 gas값을 보고 사용 단계에서 js파일을 만들 때 이용하였음에도 불구하고 오류가 해결되지 않았다. 또한 ABI도.. 분명 맞는데, 계속 gas 문제가 발생했다.  솔리리티를 삭제하고 버전도 다운시켜보고 다시 업 시켜보기도 했지만 그것도 안되었고, web3의 버전문제인가 싶어서 web3도 삭제하고 다시 깔아보았지만 해결할 수 없었다.매번 과제를 할 때마다 geth로 해보려고 온갖 노력을 해보지만, 정말 매번 도전해도 해결되지가 않는다. geth로 사용에 성공한 학우들에게 혹시 gas값을 얼마만큼 주었는지 물어보면서 비슷한 금액대로 해보아도 해결할 수가 없었다. gas오류가 떴을 때 어떻게 해결했었냐고도 물어보면 다들 안되다가 갑자기 되었다며.. 정확한 이유를 아는 학우들도 아무도 없었다. 


교수님, geth로 사용에 성공할 경우 가나슈를 이용해서 과제를 해결한 경우보다 10점을 더 주신다고 말씀하셨는데, 솔직히 많은 배점을 주시는게 아닌가 싶습니다.. 저도 게스로 성공하고 싶고 과제를 할 때마다 매번 솔리디티 버전도 바꾸어보고, 트러플인가 하는 것도 설치해보고 그랬는데도 해결되지 않습니다... geth로 성공한 친구들이 정확한 이유를 알고 geth로 성공시킨 것이라면 말이 다르겠지만, 대부분의 친구들이 갑자기 해결되었다고 말하는데, 추가 배점을 너무 크게 주는 것이 아닌지요..ㅠㅠ 평소 API가 많이 어려워서 그만큼 더 열심히 하려고 노력하는데도 정말 geth 문제는 어떻게 해야할지 잘 모르겠습니다.. 정말 gas값을 엄청 바꿔보아도 해결하지 못했어요.. 교수님께선 게스로 과제를 해결한 친구들이 더 많은 노력을 했으니 점수 배점을 크게 주시겠다고 말씀하셨는데, 가나슈로 과제를 해결한다고 해서 게스로 과제를 해결한 친구들만큼 노력을 하지 않는 게 아닙니다...ㅠㅠㅠㅠ 꼭 알아주셨으면 좋겠어요..ㅠㅠㅠㅠㅠ 